# **Camada Ouro (Modelagem Dimensional)**

Com a finalização da camada Prata, inicia-se a construção da camada Ouro, responsável por organizar os dados em estruturas analíticas otimizadas para consulta, análise e visualização. Nesta etapa, os dados já limpos e padronizados serão transformados em um modelo dimensional, seguindo o padrão snowflake, conforme definido no planejamento do projeto.

In [0]:
USE CATALOG mvp;
USE SCHEMA gold


In [0]:
CREATE OR REPLACE TABLE Fato_creditos AS
SELECT
    id AS id_titulo,
    person_id AS id_pessoa,
    role AS funcao,
    perso AS personagem
FROM mvp.silver.atores_diretores;


In [0]:
SELECT* FROM fato_creditos
LIMIT 5

In [0]:
CREATE OR REPLACE TABLE dim_titulo AS
SELECT
    s.id               AS id_titulo,
    s.title            AS titulo,
    s.category         AS show_type,
    s.release_year     AS ano_lancamento,
    s.runtime          AS duracao,
    s.seasons          AS temporadas,
    s.imdb_score,
    s.imdb_votes,
    s.tmdb_score,
    CAST(b.tmdb_popularity AS DECIMAL(10,3)) AS tmdb_popularity
FROM mvp.silver.titles s
LEFT JOIN mvp.bronze.titles b
    ON s.id = b.id;


In [0]:
COMMENT ON COLUMN dim_titulo.tmdb_popularity IS
'Métrica de popularidade do título no TMDb, utilizada como critério de desempate entre títulos com mesma pontuação.';


In [0]:
DESCRIBE TABLE dim_titulo

Devido à existência de múltiplos títulos com pontuação máxima no TMDb, a métrica de popularidade (tmdb_popularity) foi incorporada à dimensão de títulos e utilizada como critério de desempate, garantindo uma ordenação mais representativa dos conteúdos.”

In [0]:
SELECT * FROM Dim_titulo
LIMIT 5

In [0]:
CREATE OR REPLACE TABLE Dim_pessoa AS
SELECT DISTINCT
    person_id AS id_pessoa,
    name AS nome
FROM mvp.silver.atores_diretores;


In [0]:
SELECT * FROM Dim_pessoa
LIMIT 5

In [0]:
CREATE OR REPLACE TABLE Dim_genero AS
SELECT
    ROW_NUMBER() OVER (ORDER BY genres) AS id_genero,
    genres AS genero
FROM (
    SELECT DISTINCT genres
    FROM mvp.silver.titulo_genero
) g;



In [0]:
SELECT * FROM dim_genero

In [0]:
CREATE OR REPLACE TABLE Dim_pais AS
SELECT
    ROW_NUMBER() OVER (ORDER BY country) AS id_pais,
    country AS pais
FROM (
    SELECT DISTINCT country
    FROM mvp.silver.titulo_pais
) p;


In [0]:
SELECT * FROM dim_pais

In [0]:
CREATE OR REPLACE TABLE tabela_genero AS
SELECT
    tg.id_titulo,
    dg.id_genero
FROM mvp.silver.titulo_genero tg
JOIN Dim_genero dg
  ON tg.genres = dg.genero;


In [0]:
SELECT * FROM tabela_genero
LIMIT 10

In [0]:
CREATE OR REPLACE TABLE tabela_pais AS
SELECT
    tp.id AS id_titulo,
    dp.id_pais
FROM mvp.silver.titulo_pais tp
JOIN Dim_pais dp
    ON tp.country = dp.pais;


In [0]:
SELECT * FROM tabela_pais
LIMIT 10

Com a conclusão da camada Ouro, o processo de modelagem e estruturação dos dados foi finalizado com sucesso. As informações refinadas na camada Prata foram organizadas em um modelo dimensional no formato Snowflake, garantindo organização, consistência e eficiência para consultas analíticas.

Foram criadas as principais tabelas dimensão, como títulos, gêneros, países e pessoas, além das tabelas fato e de relacionamento que estruturam as associações entre essas entidades. Essa etapa consolidou os dados de maneira padronizada, permitindo análises mais rápidas, confiáveis e alinhadas às boas práticas de engenharia de dados.

A camada Ouro representa a entrega do modelo analítico pronto para consumo, possibilitando a criação de dashboards, indicadores e insights que suportam tomadas de decisão baseadas em dados.
